In [49]:
#matthew berning, 2021

import os
import sys
import math

import PIL
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

# Dataset Structure  
  
## We are interested in how the classifier performs over sucessive seasons -we have 2 to work with: 2019 and 2020  
  
so, we will train our model on 2019 data (train/test sets) and then evaluate on 2019 data and 2020 data (validation sets)

## Awned cultivars outnumber awnless cultivars considerably..

so, we need to over-sample the under represented class


In [5]:
#the 'key file' is our master dataset-structure document, it has all the file names and the respective info
key_file = pd.read_csv("/pless_nfs/home/matthewrberning/multi-year-cult-class/data/KEY_AM-PANEL_20210414.csv")

#plot_name == cultivar
#plot_id == unique plot number

key_file.head()

/pless_nfs/home/matthewrberning/miniconda3/envs/data_exp38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,plot_id,plot_name,yday,date,dir,file,PCTHEAD,AWNS,GRWT,GRYLD,LOI,MOIST,PTHT,SPKLNG,SPLSPK,SPNAREA,TESTWT,TGW
0,18ASH3-F-0100,EVEREST,128,2018-05-08,20180508_18ASH_AM3_X5R_6m_-60_video,DJI_A01733_C002_20180508_000123_1264_0824_1775...,NaN,AWNED,1216.0,2.68,NaN,14.7,67.5,NaN,NaN,NaN,71.2,NaN
1,18ASH3-F-0100,EVEREST,128,2018-05-08,20180508_18ASH_AM3_X5R_6m_-60_video,DJI_A01733_C002_20180508_000124_1776_0824_2287...,NaN,AWNED,1216.0,2.68,NaN,14.7,67.5,NaN,NaN,NaN,71.2,NaN
2,18ASH3-F-0100,EVEREST,128,2018-05-08,20180508_18ASH_AM3_X5R_6m_-60_video,DJI_A01733_C002_20180508_000123_2800_0824_3311...,NaN,AWNED,1216.0,2.68,NaN,14.7,67.5,NaN,NaN,NaN,71.2,NaN
3,18ASH3-F-0100,EVEREST,128,2018-05-08,20180508_18ASH_AM3_X5R_6m_-60_video,DJI_A01733_C002_20180508_000124_0240_0824_0751...,NaN,AWNED,1216.0,2.68,NaN,14.7,67.5,NaN,NaN,NaN,71.2,NaN
4,18ASH3-F-0100,EVEREST,128,2018-05-08,20180508_18ASH_AM3_X5R_6m_-60_video,DJI_A01733_C002_20180508_000121_2800_0824_3311...,NaN,AWNED,1216.0,2.68,NaN,14.7,67.5,NaN,NaN,NaN,71.2,NaN


### the original data (.tif files) from KSU is located inside the Globus depository

data_path: "/pless_nfs/home/matthewrberning/globus/data/KSU/images_2021_02_01/images/X5R/processed/"

we can see when we list/sort/count the dirs that there are collection events for 2018, 2019, and 2020 data

we can also see that there are more collection events for 2018 and 2019 then there are for 2020


### during the initial cultivar classification project setup the 2019 and 2020 .tif images (512x512x4) were saved as center-cropped .jpg (300x300x3)

data_path: "/pless_nfs/home/matthewrberning/multi-year-cult-class/data/preprocessed/"

this was done to increase I/O speeds while still saving some space for data augmentation (the model expects 224x224x3)

but, for safety's sake we should still validate that the data is where we think it is


In [8]:
data_path = "/pless_nfs/home/matthewrberning/globus/data/KSU/images_2021_02_01/images/X5R/processed/"
dir_lst = os.listdir(data_path)
dir_lst.sort()

#view the dirs
dir_lst

['20180508_18ASH_AM3_X5R_6m_-60_video',
 '20180510_18ASH_AM3_X5R_6m_-60_video',
 '20180514_18ASH_AM3_X5R_6m_-60_video',
 '20180515_18ASH_AM3_X5R_6m_-60_video',
 '20180516_18ASH_AM3_X5R_6m_-60_video',
 '20180517_18ASH_AM3_X5R_6m_-60_video',
 '20180518_18ASH_AM3_X5R_6m_-60_video',
 '20180521_18ASH_AM3_X5R_6m_-60_video',
 '20180522_18ASH_AM3_X5R_6m_-60_video',
 '20180523_18ASH_AM3_X5R_6m_-60_video',
 '20180525_18ASH_AM3_X5R_6m_-60_video',
 '20180528_18ASH_AM3_X5R_3m_-60_video',
 '20180529_18ASH_AM3_X5R_4m_-60_video',
 '20180531_18ASH_AM3_X5R_4m_-70_video',
 '20180601_18ASH_AM3_X5R_4m_-70_video',
 '20180613_18ASH_AM3_X5R_5m_-70_video',
 '20190506_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190508_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190510_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190512_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190513_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190515_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190516_19RF_AM_X5R_5m_-60_video_Flight1',
 '20190517_19RF_AM_X5R_5m_-60_video_Fl

### (we're working with just 2019 and 2020 for now)

In [7]:
# empty lists to collect 2019/2020 
lst_2019_dirs = []
lst_2020_dirs = []

# for each directory
for d in dir_lst:
    # check if the directory is from 2019
    if '2019' in d:
        #assemble list of 2019 dirs for training set
        lst_2019_dirs.append(d)
    
    # check if it is from 2020
    elif '2020' in d:
        # assemble list of 2020 dirs for test set
        lst_2020_dirs.append(d)


print("2019 flights: ",len(lst_2019_dirs))       
print("2020 flights: ",len(lst_2020_dirs))  

2019 flights:  16
2020 flights:  12


In [48]:
#count and display the number of images in each directory
previous_year = 2018 #var to help seperate out years

year_count = 0 #var to track total images in a year

dir_count = 0

for d in dir_lst:
    
    #var to track the number of images
    count = 0 
    
    #var to print the year
    if '2018' in d:
        year = 2018
    elif '2019' in d:
        year = 2019
    else:
        year = 2020
        
    #loop through list of objects inside the specified directory
    for obj in os.listdir(os.path.join(data_path, d)):

        #if it's a tif add to count
        if obj.endswith('.tif'):
            count+=1
            
    #align numbers for sanity
    count = str(count).rjust(7)
    
    #seperate out years
    if year != previous_year:
        #report the total and average
        print(f"-       TOTAL: {str(year_count).rjust(7)}   dirs: {dir_count} [ average: {math.ceil(year_count/dir_count)} ]")
        year_count = 0 #reset the year count
        
        dir_count = 0 #reset the dir count
    
    #reset the current previous year
    previous_year = year
    
    print(f"{year} | images: {count}   dir: {d}")
    
    #add the count for that dir to the total for the year
    year_count+=int(count)
    
    #add to the count of directories for that year
    dir_count+=1

#report
print(f"-       TOTAL: {str(year_count).rjust(7)}   dirs: {dir_count} [ average: {math.ceil(year_count/dir_count)} ]")

2018 | images:   55783   dir: 20180508_18ASH_AM3_X5R_6m_-60_video
2018 | images:   27230   dir: 20180510_18ASH_AM3_X5R_6m_-60_video
2018 | images:   60368   dir: 20180514_18ASH_AM3_X5R_6m_-60_video
2018 | images:   60326   dir: 20180515_18ASH_AM3_X5R_6m_-60_video
2018 | images:   97461   dir: 20180516_18ASH_AM3_X5R_6m_-60_video
2018 | images:   55503   dir: 20180517_18ASH_AM3_X5R_6m_-60_video
2018 | images:   60368   dir: 20180518_18ASH_AM3_X5R_6m_-60_video
2018 | images:   50512   dir: 20180521_18ASH_AM3_X5R_6m_-60_video
2018 | images:   56861   dir: 20180522_18ASH_AM3_X5R_6m_-60_video
2018 | images:   88319   dir: 20180523_18ASH_AM3_X5R_6m_-60_video
2018 | images:   55258   dir: 20180525_18ASH_AM3_X5R_6m_-60_video
2018 | images:   66682   dir: 20180528_18ASH_AM3_X5R_3m_-60_video
2018 | images:   32116   dir: 20180529_18ASH_AM3_X5R_4m_-60_video
2018 | images:  106197   dir: 20180531_18ASH_AM3_X5R_4m_-70_video
2018 | images:   64596   dir: 20180601_18ASH_AM3_X5R_4m_-70_video
2018 | ima

### now we'll isolate the image file names from 2019 and 2020 in the three places they're located
1 - the key file  
2 - the globus depository (.tif)  
3 - the dir with cropped images (.jpg)  


In [67]:
#path to cropped data
data_path = "/pless_nfs/home/matthewrberning/multi-year-cult-class/data/preprocessed/"

#collect all the file names if they are images (end in .jpg)
cropped_filenames_list = []

path_lst = os.listdir(data_path)

for obj in path_lst:
    
    #check if it's the image type we're looking for
    if obj.endswith(".jpg"):
        cropped_filenames_list.append(obj)
        
print(len(cropped_filenames_list), " image file names found")        
print("example: ", cropped_filenames_list[0])

2227833  image file names found
example:  20190522_19RF_AM_X5R_5m_-60_video_Flight1_DJI_A00417_C024_20190522_000319_3312_0824_3823_1336_19RKY00116.jpg


In [60]:
#path to globus data
data_path = "/pless_nfs/home/matthewrberning/globus/data/KSU/images_2021_02_01/images/X5R/processed/"

#use previously found 2019 and 2020 dir names (lst_2019_dirs and lst_2020_dirs)
list_of_dirs = lst_2019_dirs + lst_2020_dirs

#empty list for collecting image paths
tif_filenames_list = []


#loop through list of dirs
for _dir in tqdm(list_of_dirs):

    #loop through list of objects inside the specified directory
    for obj in os.listdir(os.path.join(data_path, _dir)):

        #it's probably an image, add it to the list
        if obj.endswith('.tif'):
            
            #modify the filename to match those in jpg dir for easier comparison
            tif_filenames_list.append(f"{_dir}_{obj}")

print(len(tif_filenames_list), " image file names (plus dir name) found")        
print("example: ", tif_filenames_list[0])

100%|██████████| 28/28 [00:02<00:00, 12.23it/s]

2360441  image file names (plus dir name) found
example:  20190506_19RF_AM_X5R_5m_-60_video_Flight1_DJI_A06276_C005_20190506_000722_1776_0824_2287_1336_19RKY00623.tif


In [71]:
#collect from the key_file
key_file_filenames_list = []

#copy the relevant data into a new df to collect from the key_file
#(remember this doesn't work with mutable objects in a df)
key_file_filenames_df = key_file[['dir', 'file']].copy(deep=True) 

#create a funt to combine path info into the dataframe too 
def create_filename(row, path):
    
    #concat the info in the rows
    filename = row['dir'] + '_' + row['file']

    return filename

#apply the abofe helper function with lambda (expect a 'SettingWithCopyWarning')
key_file_filenames_df['img_loc'] = key_file_filenames_df.apply (lambda row: create_filename(row, path), axis=1)

#turn the whole collumn (2018, 2019, and 2020 names included) into a list
key_file_filenames_lst_all = key_file_filenames_df['img_loc'].tolist()

#get only 2019 and 2020 filenames
for name in key_file_filenames_lst_all:
    if '2019' in name or '2020' in name:
        key_file_filenames_list.append(name)

print(len(key_file_filenames_lst_all), " total image filenames")
print(len(key_file_filenames_list), " image file names (plus dir name) gotten from key_file")        
print("example: ", key_file_filenames_list[0])

3410105  total image filenames
2360448  image file names (plus dir name) gotten from key_file
example:  20190506_19RF_AM_X5R_5m_-60_video_Flight1_DJI_A06276_C028_20190506_000040_1776_0824_2287_1336_19RKY0-F-0100.tif


In [ ]:
#helper functions to compare lists
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    return lst3

print(intersection(tif_filenames_list, key_file_filenames_list))